# TriKirby Index

### The TriKirby Index is a methodology inspired by the Kirby Index that quantifies the command and effectiveness of each UCSD Pitcher's pitches ahead of the 2026 season.

## TriKirby Index – Equation & Components

The **TriKirby Index** measures pitcher command via mechanical repeatability, evaluated separately for each pitch type and normalized against all NCAA Division I pitchers, then aggregated using pitch usage.

---

### 1. Release Angle Variability

For each **UCSD pitcher** \( p \) and pitch type \( t \):

- $\sigma_{\text{HRA},p,t}$: Standard deviation of **Horizontal Release Angle**
- $\sigma_{\text{VRA},p,t}$: Standard deviation of **Vertical Release Angle**

Lower variability indicates better command and mechanical repeatability.

---

### 2. NCAA-Wide Normalization (Z-Scores)

Each UCSD pitcher’s release variability is normalized using **all NCAA Division I pitchers**:

$$
z^{(k)}_{p,t}
=
\frac{
\sigma^{(k)}_{p,t} - \mu^{(k)}_{\text{NCAA}}
}{
\sigma^{(k)}_{\text{NCAA}}
},
\quad
k \in \{\text{HRA}, \text{VRA}\}
$$

Where:

- $\mu^{(k)}_{\text{NCAA}}$: Mean variability across all NCAA Division I pitchers  
- $\sigma^{(k)}_{\text{NCAA}}$: Standard deviation across all NCAA Division I pitchers  

**Negative values indicate better-than-average command relative to NCAA Division I.**

---

### 3. Pitch-Level TriKirby Score

For each UCSD pitcher and pitch type, the pitch-level TriKirby score is defined as:

$$
\text{TriKirby}_{p,t}
=
\frac{
z^{\text{HRA}}_{p,t} + z^{\text{VRA}}_{p,t}
}{2}
$$

This score captures overall release-angle consistency for a specific pitch.

---

### 4. Pitcher-Level TriKirby Index (Usage-Weighted)

To evaluate overall command for a **UCSD pitcher**, pitch-level scores are aggregated using pitch usage:

$$
\text{TriKirby Index}_p
=
\sum_t
\left(
\text{TriKirby}_{p,t}
\times
\text{Usage}_{p,t}
\right)
$$

Where $\text{Usage}_{p,t}$ represents the proportion of pitch type \( t \) thrown by pitcher \( p \).

---

### 5. Interpretation

| TriKirby Value | Interpretation (Relative to NCAA Division I) |
|---------------|----------------------------------------------|
| **< 0** | Above-average command |
| **≈ 0** | NCAA-average command |
| **> 0** | Below-average command |
| **More negative** | Elite release consistency |

# Import Libraries + Dataset Upload

In [1]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("Trackman CSVs")   # matches folder name exactly

csv_paths = sorted(DATA_DIR.glob("*.csv"))
print("CSV files found:", len(csv_paths))

dfs = []
for p in csv_paths:
    try:
        df = pd.read_csv(p)
        df["source_file"] = p.name
        dfs.append(df)
    except Exception as e:
        print(f"Skipped {p.name}: {e}")

all_df = pd.concat(dfs, ignore_index=True)

print("Merged rows:", len(all_df))
print("Merged cols:", len(all_df.columns))

all_df.head()

CSV files found: 974
Merged rows: 297652
Merged cols: 171


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,source_file,Runner1st,Runner2nd,Runner3rd
0,1,2025-02-14,18:04:47.71,1,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,High,NaN,NaN,NaN,NaN,NaN,20250214-AlumniField-1.csv,NaN,NaN,NaN
1,2,2025-02-14,18:05:00.16,1,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,High,NaN,NaN,NaN,NaN,NaN,20250214-AlumniField-1.csv,NaN,NaN,NaN
2,3,2025-02-14,18:05:14.48,1,3,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,High,NaN,NaN,Medium,Medium,Low,20250214-AlumniField-1.csv,NaN,NaN,NaN
3,4,2025-02-14,18:05:39.46,2,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Edwards, Kameron",...,High,NaN,NaN,NaN,NaN,NaN,20250214-AlumniField-1.csv,NaN,NaN,NaN
4,5,2025-02-14,18:05:53.55,2,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Edwards, Kameron",...,High,NaN,NaN,NaN,NaN,NaN,20250214-AlumniField-1.csv,NaN,NaN,NaN


In [2]:
# Sometimes Trackman exports slightly different columns.
dfs = []
all_cols = set()

# first pass: collect columns
for p in csv_paths:
    df = pd.read_csv(p)
    all_cols |= set(df.columns)

all_cols = sorted(all_cols)

# second pass: align columns
for p in csv_paths:
    df = pd.read_csv(p)
    df = df.reindex(columns=all_cols)  # missing cols become NaN
    df["source_file"] = p.name
    dfs.append(df)

all_df = pd.concat(dfs, ignore_index=True)

In [3]:
# Access a single CSV
# df = pd.read_csv("data/2025/20250214-AlumniField-1.csv")

In [4]:
# Make sure all components of the TriKirby Index is accessable 
needed = ["Pitcher", "TaggedPitchType", "HorzRelAngle", "VertRelAngle"]
missing = [c for c in needed if c not in all_df.columns]
missing

[]

# TriKirby Calculation Set-Up

In [5]:
# Only using the columns we need to quantify the Kirby Index score for all pitchers in college baseball.
KEEP_COLS = [
    "Pitcher",              # or PitcherName / PitcherId
    "AutoPitchType",
    "TaggedPitchType",
    "HorzRelAngle",
    "VertRelAngle"
]

df = all_df[KEEP_COLS].copy()
df = df.dropna()

print("Rows after filtering:", len(df))
df.head()


Rows after filtering: 294052


,Pitcher,AutoPitchType,TaggedPitchType,HorzRelAngle,VertRelAngle
0,"Stuprich, Brennan",Four-Seam,Undefined,-2.909177,-3.031037
1,"Stuprich, Brennan",Four-Seam,Undefined,-3.304661,-2.645502
2,"Stuprich, Brennan",Four-Seam,Undefined,-2.887164,-1.756479
3,"Stuprich, Brennan",Four-Seam,Undefined,-2.187817,-2.143004
4,"Stuprich, Brennan",Four-Seam,Undefined,-1.858974,-3.768853


In [6]:
# Adding a minimum amount of pitches measured to reduce noise
MIN_PITCHES = 40

df = (
    df.groupby(["Pitcher", "TaggedPitchType"])
      .filter(lambda x: len(x) >= MIN_PITCHES)
)

print("Rows after min pitch filter:", len(df))

Rows after min pitch filter: 134184


# TriKirby Index Calculation: Command = How tightly a pitcher repeats release angles (lower spread = better command)

In [7]:
import numpy as np

def trikirby_raw(group):
    h_sd = group["HorzRelAngle"].std(ddof=1)
    v_sd = group["VertRelAngle"].std(ddof=1)
    return np.sqrt(h_sd**2 + v_sd**2)

In [8]:
trikirby = (
    df.groupby(["Pitcher", "TaggedPitchType"])
      .apply(trikirby_raw)
      .reset_index(name="trikirby_raw")
)

trikirby.head()

/var/folders/zh/lt350y8j697d9gr_ryq4y0gm0000gn/T/ipykernel_3602/4221991766.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(trikirby_raw)


,Pitcher,TaggedPitchType,trikirby_raw
0,"Abell, Mark",Fastball,1.454119
1,"Aceves, Nathan",Fastball,1.435912
2,"Adams, George",Fastball,1.475173
3,"Adams, Matthew",Fastball,1.158601
4,"Adams, Zane",Fastball,1.194723


In [9]:
# Normalize by pitch type: Lower z-score = better command
trikirby["trikirby_z"] = (
    trikirby
    .groupby("TaggedPitchType")["trikirby_raw"]
    .transform(lambda x: (x - x.mean()) / x.std(ddof=1))
)

In [10]:
usage = (
    df.groupby(["Pitcher", "TaggedPitchType"])
      .size()
      .reset_index(name="n")
)

usage["usage_pct"] = usage.groupby("Pitcher")["n"].transform(lambda x: x / x.sum())

trikirby = trikirby.merge(
    usage[["Pitcher", "TaggedPitchType", "usage_pct"]],
    on=["Pitcher", "TaggedPitchType"]
)

overall = (
    trikirby
    .assign(weighted = trikirby["trikirby_z"] * trikirby["usage_pct"])
    .groupby("Pitcher")["weighted"]
    .sum()
    .reset_index(name="TriKirby_Index")
    .sort_values("TriKirby_Index")
)
overall.head(10)

,Pitcher,TriKirby_Index
1272,"Taylor, Zane",-2.119996
972,"Parmer, Evan",-1.958310
530,"Hart, Zach",-1.910339
910,"Murray, Landon",-1.869241
32,"Applebey, Kaleb",-1.803135
1312,"Tuttoilmondo, Joey",-1.734943
121,"Boyer, Matt",-1.708552
653,"Kelly, Lucas",-1.629109
659,"Kennedy, Jay",-1.605716
805,"Martinez, Ryan",-1.605508


In [12]:
overall.describe()

,TriKirby_Index
count,1414.000000
mean,-0.015179
std,0.925168
min,-2.119996
25%,-0.549438
50%,-0.104641
75%,0.385488
max,8.996258


# Build NCAA-wide TriKirby (pitcher × pitch type), then z-score by pitch type

In [13]:
import pandas as pd
import numpy as np

# --- assumes your merged dataframe is named: all_df ---
df = all_df.copy()

# Columns your TriKirby uses
REQ = ["Pitcher", "TaggedPitchType", "HorzRelAngle", "VertRelAngle"]
missing = [c for c in REQ if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# (Optional but recommended) drop rows missing angles / identifiers
df = df.dropna(subset=["Pitcher", "TaggedPitchType", "HorzRelAngle", "VertRelAngle"])

# --- Raw TriKirby per (Pitcher, PitchType): lower spread = better command ---
def trikirby_raw(group):
    h_sd = group["HorzRelAngle"].std(ddof=1)
    v_sd = group["VertRelAngle"].std(ddof=1)
    return np.sqrt(h_sd**2 + v_sd**2)

trikirby = (
    df.groupby(["Pitcher", "TaggedPitchType"], as_index=False)
      .apply(lambda g: pd.Series({"trikirby_raw": trikirby_raw(g)}))
)

# NCAA-wide normalization (BY PITCH TYPE): lower z = better command
trikirby["trikirby_z"] = (
    trikirby.groupby("TaggedPitchType")["trikirby_raw"]
           .transform(lambda x: (x - x.mean()) / x.std(ddof=1))
)

# Optional: a "positive = better" version (so higher = better command)
trikirby["trikirby_pos"] = -trikirby["trikirby_z"]

trikirby.head()


/var/folders/zh/lt350y8j697d9gr_ryq4y0gm0000gn/T/ipykernel_3602/3654757598.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({"trikirby_raw": trikirby_raw(g)}))


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Abad, Adrian",Sinker,0.738072,-1.347418,1.347418
1,"Abad, Adrian",Slider,1.697732,0.370080,-0.370080
2,"Abbadessa, Jude",ChangeUp,0.678526,-1.519942,1.519942
3,"Abbadessa, Jude",Fastball,1.093046,-0.862790,0.862790
4,"Abbadessa, Jude",Slider,1.362142,-0.361198,0.361198


# UC San Diego Triton Pitchers: TriKirby Index Score for Each Pitch

## Identify Our Pitchers from the Trackman CSV Files

In [14]:
# Give our pitchers a code to easily identify each pitcher
UCSD_TEAM_CODE = "CSD_TRI"

ucsd_trikirby = trikirby[trikirby["Pitcher"].isin(
    df[df["PitcherTeam"] == UCSD_TEAM_CODE]["Pitcher"].unique()
)].copy()

print("UCSD pitchers in TriKirby:", ucsd_trikirby["Pitcher"].nunique())


UCSD pitchers in TriKirby: 20


In [15]:
ucsd_pitchers = (
    df[df["PitcherTeam"] == "CSD_TRI"]["Pitcher"]
    .dropna()
    .unique()
)

print("Number of UCSD pitchers:", len(ucsd_pitchers))
sorted(ucsd_pitchers)

Number of UCSD pitchers: 20


['Cazares, Julian',
 'Custer, Julian',
 'Dalquist, Matthew',
 'Davidson, Garrett',
 'Ernisse, Zach',
 'Gregson, Niccolas',
 'Hasegawa, Sam',
 'Huy, Nathan',
 'King, Devon',
 'Marchetti, Landon',
 'Murdock, Steele',
 'Nickerson, Trevor',
 'Patterson, Garrett',
 'Pelzman, Harry',
 'Rector, Trevor',
 'Remmers, Ethan',
 'Ries, Nathan',
 'Seid, Spencer',
 'Villar, Jake',
 'Weber, Chapman']

# Build UCSD-only TriKirby table 

In [16]:
# UCSD filter (update if your code changes the team column name)
UCSD_TEAM_CODE = "CSD_TRI"
TEAM_COL = "PitcherTeam"   # change only if your column name is different

# list of UCSD pitchers from the full raw pitch dataset (df)
ucsd_pitchers = (
    df.loc[df[TEAM_COL].astype(str).str.strip() == UCSD_TEAM_CODE, "Pitcher"]
      .dropna()
      .unique()
)

print("UCSD pitchers found:", len(ucsd_pitchers))

# IMPORTANT: use the NCAA-normalized trikirby table you already built earlier
# trikirby must contain: Pitcher, TaggedPitchType, trikirby_raw, trikirby_z (and/or trikirby_pos)
ucsd_trikirby = trikirby[trikirby["Pitcher"].isin(ucsd_pitchers)].copy()

print("Rows in UCSD TriKirby table:", len(ucsd_trikirby))
print("Pitch types present (UCSD):", sorted(ucsd_trikirby["TaggedPitchType"].dropna().unique()))
ucsd_trikirby.head()

UCSD pitchers found: 20
Rows in UCSD TriKirby table: 105
Pitch types present (UCSD): ['ChangeUp', 'Curveball', 'Cutter', 'Fastball', 'FourSeamFastBall', 'Knuckleball', 'Sinker', 'Slider', 'Splitter', 'Sweeper', 'TwoSeamFastBall']


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
2208,"Cazares, Julian",ChangeUp,1.299545,-0.304470,0.304470
2209,"Cazares, Julian",Cutter,NaN,NaN,NaN
2210,"Cazares, Julian",Fastball,1.261693,-0.271035,0.271035
2211,"Cazares, Julian",FourSeamFastBall,2.209184,2.778803,-2.778803
2212,"Cazares, Julian",Sinker,NaN,NaN,NaN


# TriKirby Index Score for Each Pitch Type (UC San Diego Triton Pitching)

In [17]:
# Helper function to display a clean per-pitch table
def show_ucsd_pitch_table(pitch_type, sort_col="trikirby_z", top_n=200):
    out = (
        ucsd_trikirby.loc[
            ucsd_trikirby["TaggedPitchType"].astype(str).str.lower() == str(pitch_type).lower(),
            ["Pitcher", "TaggedPitchType", "trikirby_raw", "trikirby_z"]
            + (["trikirby_pos"] if "trikirby_pos" in ucsd_trikirby.columns else [])
        ]
        .sort_values(sort_col, ascending=True)  # lower z = tighter angles = better command
        .reset_index(drop=True)
    )
    print(f"\nUCSD — {pitch_type} (sorted by {sort_col}; lower = better command)")
    display(out.head(top_n))
    return out

In [18]:
show_ucsd_pitch_table("Fastball")


UCSD — Fastball (sorted by trikirby_z; lower = better command)


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Patterson, Garrett",Fastball,0.998757,-1.193636,1.193636
1,"Custer, Julian",Fastball,1.188230,-0.528806,0.528806
2,"Dalquist, Matthew",Fastball,1.201962,-0.480623,0.480623
3,"Cazares, Julian",Fastball,1.261693,-0.271035,0.271035
4,"Villar, Jake",Fastball,1.322894,-0.056291,0.056291
5,"Ries, Nathan",Fastball,1.326117,-0.044981,0.044981
6,"Murdock, Steele",Fastball,1.330267,-0.030422,0.030422
7,"Seid, Spencer",Fastball,1.346629,0.026992,-0.026992
8,"King, Devon",Fastball,1.361580,0.079452,-0.079452
9,"Gregson, Niccolas",Fastball,1.363512,0.086233,-0.086233


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Patterson, Garrett",Fastball,0.998757,-1.193636,1.193636
1,"Custer, Julian",Fastball,1.188230,-0.528806,0.528806
2,"Dalquist, Matthew",Fastball,1.201962,-0.480623,0.480623
3,"Cazares, Julian",Fastball,1.261693,-0.271035,0.271035
4,"Villar, Jake",Fastball,1.322894,-0.056291,0.056291
5,"Ries, Nathan",Fastball,1.326117,-0.044981,0.044981
6,"Murdock, Steele",Fastball,1.330267,-0.030422,0.030422
7,"Seid, Spencer",Fastball,1.346629,0.026992,-0.026992
8,"King, Devon",Fastball,1.361580,0.079452,-0.079452
9,"Gregson, Niccolas",Fastball,1.363512,0.086233,-0.086233


In [19]:
show_ucsd_pitch_table("Changeup")


UCSD — Changeup (sorted by trikirby_z; lower = better command)


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"King, Devon",ChangeUp,0.980545,-0.928824,0.928824
1,"Dalquist, Matthew",ChangeUp,1.289056,-0.325000,0.325000
2,"Cazares, Julian",ChangeUp,1.299545,-0.304470,0.304470
3,"Pelzman, Harry",ChangeUp,1.305954,-0.291927,0.291927
4,"Remmers, Ethan",ChangeUp,1.329307,-0.246219,0.246219
5,"Davidson, Garrett",ChangeUp,1.350164,-0.205398,0.205398
6,"Marchetti, Landon",ChangeUp,1.407023,-0.094112,0.094112
7,"Villar, Jake",ChangeUp,1.408362,-0.091491,0.091491
8,"Murdock, Steele",ChangeUp,1.467611,0.024472,-0.024472
9,"Seid, Spencer",ChangeUp,1.596762,0.277248,-0.277248


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"King, Devon",ChangeUp,0.980545,-0.928824,0.928824
1,"Dalquist, Matthew",ChangeUp,1.289056,-0.325000,0.325000
2,"Cazares, Julian",ChangeUp,1.299545,-0.304470,0.304470
3,"Pelzman, Harry",ChangeUp,1.305954,-0.291927,0.291927
4,"Remmers, Ethan",ChangeUp,1.329307,-0.246219,0.246219
5,"Davidson, Garrett",ChangeUp,1.350164,-0.205398,0.205398
6,"Marchetti, Landon",ChangeUp,1.407023,-0.094112,0.094112
7,"Villar, Jake",ChangeUp,1.408362,-0.091491,0.091491
8,"Murdock, Steele",ChangeUp,1.467611,0.024472,-0.024472
9,"Seid, Spencer",ChangeUp,1.596762,0.277248,-0.277248


In [20]:
show_ucsd_pitch_table("Slider")


UCSD — Slider (sorted by trikirby_z; lower = better command)


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Dalquist, Matthew",Slider,1.235841,-0.636420,0.636420
1,"Huy, Nathan",Slider,1.308547,-0.477987,0.477987
2,"Ries, Nathan",Slider,1.450114,-0.169501,0.169501
3,"Murdock, Steele",Slider,1.469099,-0.128131,0.128131
4,"Gregson, Niccolas",Slider,1.504284,-0.051460,0.051460
5,"Patterson, Garrett",Slider,1.538125,0.022284,-0.022284
6,"Seid, Spencer",Slider,1.539008,0.024206,-0.024206
7,"King, Devon",Slider,1.600495,0.158191,-0.158191
8,"Weber, Chapman",Slider,1.639849,0.243947,-0.243947
9,"Davidson, Garrett",Slider,1.653310,0.273280,-0.273280


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Dalquist, Matthew",Slider,1.235841,-0.636420,0.636420
1,"Huy, Nathan",Slider,1.308547,-0.477987,0.477987
2,"Ries, Nathan",Slider,1.450114,-0.169501,0.169501
3,"Murdock, Steele",Slider,1.469099,-0.128131,0.128131
4,"Gregson, Niccolas",Slider,1.504284,-0.051460,0.051460
5,"Patterson, Garrett",Slider,1.538125,0.022284,-0.022284
6,"Seid, Spencer",Slider,1.539008,0.024206,-0.024206
7,"King, Devon",Slider,1.600495,0.158191,-0.158191
8,"Weber, Chapman",Slider,1.639849,0.243947,-0.243947
9,"Davidson, Garrett",Slider,1.653310,0.273280,-0.273280


In [21]:
show_ucsd_pitch_table("Curveball")


UCSD — Curveball (sorted by trikirby_z; lower = better command)


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"King, Devon",Curveball,1.224118,-0.666921,0.666921
1,"Dalquist, Matthew",Curveball,1.296532,-0.537469,0.537469
2,"Marchetti, Landon",Curveball,1.356954,-0.429453,0.429453
3,"Gregson, Niccolas",Curveball,1.660217,0.112679,-0.112679
4,"Hasegawa, Sam",Curveball,1.967714,0.662380,-0.662380
5,"Rector, Trevor",Curveball,2.068561,0.842661,-0.842661
6,"Ries, Nathan",Curveball,2.115578,0.926711,-0.926711
7,"Davidson, Garrett",Curveball,2.145118,0.979519,-0.979519
8,"Villar, Jake",Curveball,2.664579,1.908140,-1.908140
9,"Murdock, Steele",Curveball,NaN,NaN,NaN


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"King, Devon",Curveball,1.224118,-0.666921,0.666921
1,"Dalquist, Matthew",Curveball,1.296532,-0.537469,0.537469
2,"Marchetti, Landon",Curveball,1.356954,-0.429453,0.429453
3,"Gregson, Niccolas",Curveball,1.660217,0.112679,-0.112679
4,"Hasegawa, Sam",Curveball,1.967714,0.662380,-0.662380
5,"Rector, Trevor",Curveball,2.068561,0.842661,-0.842661
6,"Ries, Nathan",Curveball,2.115578,0.926711,-0.926711
7,"Davidson, Garrett",Curveball,2.145118,0.979519,-0.979519
8,"Villar, Jake",Curveball,2.664579,1.908140,-1.908140
9,"Murdock, Steele",Curveball,NaN,NaN,NaN


In [23]:
show_ucsd_pitch_table("Sinker")


UCSD — Sinker (sorted by trikirby_z; lower = better command)


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Rector, Trevor",Sinker,0.887108,-0.964127,0.964127
1,"Remmers, Ethan",Sinker,1.065535,-0.505248,0.505248
2,"Villar, Jake",Sinker,1.172748,-0.229517,0.229517
3,"Murdock, Steele",Sinker,1.201489,-0.155601,0.155601
4,"Cazares, Julian",Sinker,NaN,NaN,NaN
5,"Custer, Julian",Sinker,NaN,NaN,NaN
6,"Hasegawa, Sam",Sinker,NaN,NaN,NaN
7,"Ries, Nathan",Sinker,NaN,NaN,NaN


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Rector, Trevor",Sinker,0.887108,-0.964127,0.964127
1,"Remmers, Ethan",Sinker,1.065535,-0.505248,0.505248
2,"Villar, Jake",Sinker,1.172748,-0.229517,0.229517
3,"Murdock, Steele",Sinker,1.201489,-0.155601,0.155601
4,"Cazares, Julian",Sinker,NaN,NaN,NaN
5,"Custer, Julian",Sinker,NaN,NaN,NaN
6,"Hasegawa, Sam",Sinker,NaN,NaN,NaN
7,"Ries, Nathan",Sinker,NaN,NaN,NaN


In [24]:
show_ucsd_pitch_table("Splitter")


UCSD — Splitter (sorted by trikirby_z; lower = better command)


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Seid, Spencer",Splitter,1.772729,0.396946,-0.396946


,Pitcher,TaggedPitchType,trikirby_raw,trikirby_z,trikirby_pos
0,"Seid, Spencer",Splitter,1.772729,0.396946,-0.396946
